## Initial Setup

1. Download the Zeus 1 version 1.2.7.19 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/0060fa563c86399ac56dfc261181beeeafc3a74ded1f88ee248d794fcb14e178/).
2. Unpack the sample with [PE-sieve](https://github.com/hasherezade/pe-sieve).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import pprint
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("c049fe5f480e0f676533b30d4ba00053d9329369093c1c212bb74e64a7f1ec22.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Resolve Windows API Calls

Pseudocode:

    bool resolve_all_win_apis()
    {
      kernel32_hdl = resolve_win_apis("kernel32.dll", &kernel32_apis, 125u);
    ...
    
Disassembly:

    .text:00409C37 6A 7D                         push    7Dh ; '}'
    .text:00409C39 BA 08 12 40 00                mov     edx, offset kernel32_apis
    .text:00409C3E B9 50 38 40 00                mov     ecx, offset aKernel32Dll ; "kernel32.dll"
    .text:00409C43 E8 CA 09 00 00                call    resolve_win_apis
    .text:00409C48 A3 38 4D 41 00                mov     kernel32_hdl, eax
    ...
    
Pseudocode:

    HMODULE __fastcall resolve_win_apis(const CHAR *dll_name, int api_list, unsigned __int16 num_entries)
    {
      HMODULE dll_hdl; // ebp
      unsigned __int16 index; // di
      int offset; // esi
      FARPROC api_addr; // eax

      dll_hdl = LoadLibraryA(dll_name);
      if ( !dll_hdl )
        return dll_hdl;
      index = 0;
      if ( !num_entries )
        return dll_hdl;
      while ( 1 )
      {
        offset = api_list + 8 * index;
        api_addr = GetProcAddress(dll_hdl, *offset);
        if ( !api_addr )
          break;
        ++index;
        **(offset + 4) = api_addr;
        if ( index >= num_entries )
          return dll_hdl;
      }
      return 0;
    }

In [3]:
match = re.search(rb'\x6a(?P<num_entries>\x7d)\xba(?P<api_list_addr>[\s\S]{4})', pe.get_memory_mapped_image())
num_entries = struct.unpack("B", match.group("num_entries"))[0]
api_list_addr = struct.unpack("I", match.group("api_list_addr"))[0]

for i in range(num_entries):
    offset = api_list_addr + 8 * i
    offset_rva = offset - pe.OPTIONAL_HEADER.ImageBase
    api_name_addr = pe.get_dword_at_rva(offset_rva)
    api_name_addr_rva = api_name_addr - pe.OPTIONAL_HEADER.ImageBase
    api_name = pe.get_data(api_name_addr_rva, 128).split(b"\x00")[0].decode()
    print(f"{i}: {api_name}")

0: FormatMessageW
1: LocalFree
2: GetEnvironmentVariableW
3: GetOverlappedResult
4: GetVolumeNameForVolumeMountPointW
5: WaitForMultipleObjects
6: GetFileInformationByHandle
7: FileTimeToLocalFileTime
8: FileTimeToDosDateTime
9: CreateTimerQueueTimer
10: GetThreadContext
11: SetThreadContext
12: LoadLibraryA
13: GetProcAddress
14: ResumeThread
15: OpenThread
16: FreeLibrary
17: SuspendThread
18: GetProcessId
19: GetFileAttributesW
20: GetProcessHeap
21: GetCommandLineA
22: GetLogicalDrives
23: GetDriveTypeW
24: GetFileSizeEx
25: CreateFileMappingW
26: MapViewOfFile
27: UnmapViewOfFile
28: ResetEvent
29: GetTimeZoneInformation
30: GetVersionExW
31: GetUserDefaultUILanguage
32: GetModuleHandleA
33: Sleep
34: CopyFileW
35: GetModuleFileNameW
36: GetModuleFileNameA
37: SetFilePointerEx
38: GetLastError
39: CreateMutexW
40: OpenMutexW
41: ReleaseMutex
42: lstrcmpiW
43: lstrcmpiA
44: lstrcpyW
45: lstrcpynW
46: lstrcpynA
47: lstrcpyA
48: lstrcatW
49: lstrcatA
50: CloseHandle
51: DeleteFileW
5

## Decrypt Strings

Pseudocode:

    ...
    g_decrypted_strings = HeapAlloc(hHeap, 8u, 416u);
    if ( g_decrypted_strings )
    {
      index = 0;
      do
      {
        offset = 4 * index;
        if ( **(&g_encrypted_strings + offset) == -1 )
          return 0;
        buf = Mem::alloc(**(&g_encrypted_strings + index) + 1);
        if ( !buf )
          return 0;
        encrypted_string_entry = *(&g_encrypted_strings + index);
        v7 = *encrypted_string_entry <= 0;
        buf_index_cp = 0;
        if ( !v7 )
        {
          buf_index = 0;
          key = 0xBA;
          do
          {
            plain_byte = key + encrypted_string_entry[buf_index + 1];
            ++buf_index_cp;
            buf[buf_index] = plain_byte;
            buf_index = buf_index_cp;
            key += 2;
          }
          while ( buf_index_cp < *encrypted_string_entry );
        }
        buf[buf_index_cp] = 0;
        if ( *buf == 'W' )
        {
          *(offset + g_decrypted_strings) = Str::xToUnicode(buf + 1);
          Mem::free(buf);
          if ( !*(offset + g_decrypted_strings) )
            return 0;
        }
        else
        {
          *(offset + g_decrypted_strings) = buf + 1;
        }
        ++index;
      }
      while ( index < 103u );
      ...

Disassembly:

    ...
    .text:0040A725 68 A0 01 00 00                                push    1A0h
    .text:0040A72A 6A 08                                         push    8
    .text:0040A72C FF 35 C4 5F 41 00                             push    hHeap
    .text:0040A732 89 35 C4 4F 41 00                             mov     dword_414FC4, esi
    .text:0040A738 C6 05 DE 47 41 00 00                          mov     byte_4147DE, 0
    .text:0040A73F FF 15 2C 4E 41 00                             call    HeapAlloc
    .text:0040A745 A3 F4 4A 41 00                                mov     g_decrypted_strings, eax
    .text:0040A74A 3B C6                                         cmp     eax, esi
    .text:0040A74C 75 07                                         jnz     short loc_40A755
    .text:0040A74E 32 C0                                         xor     al, al
    .text:0040A750 E9 5B 01 00 00                                jmp     loc_40A8B0
    .text:0040A755
    .text:0040A755                               loc_40A755:
    .text:0040A755 89 75 F8                                      mov     [ebp+index], esi
    .text:0040A758 57                                            push    edi
    .text:0040A759
    .text:0040A759                               loc_40A759:
    .text:0040A759 0F B7 5D F8                                   movzx   ebx, word ptr [ebp+index]
    .text:0040A75D C1 E3 02                                      shl     ebx, 2
    .text:0040A760 8B 83 20 40 41 00                             mov     eax, g_encrypted_strings[ebx]
    .text:0040A766 0F B6 00                                      movzx   eax, byte ptr [eax]
    .text:0040A769 40                                            inc     eax
    .text:0040A76A 74 72                                         jz      short loc_40A7DE
    .text:0040A76C E8 60 44 00 00                                call    Mem__alloc
    .text:0040A771 8B F8                                         mov     edi, eax
    .text:0040A773 85 FF                                         test    edi, edi
    .text:0040A775 74 67                                         jz      short loc_40A7DE
    .text:0040A777 8B B3 20 40 41 00                             mov     esi, g_encrypted_strings[ebx]
    .text:0040A77D 80 3E 00                                      cmp     byte ptr [esi], 0
    .text:0040A780 C6 45 FF 00                                   mov     [ebp+buf_index_cp], 0
    .text:0040A784 7E 1E                                         jle     short loc_40A7A4
    .text:0040A786 33 C0                                         xor     eax, eax
    .text:0040A788 B1 BA                                         mov     cl, 0BAh
    .text:0040A78A
    .text:0040A78A                               loc_40A78A:
    .text:0040A78A 8A 54 30 01                                   mov     dl, [eax+esi+1]
    .text:0040A78E 02 D1                                         add     dl, cl
    .text:0040A790 FE 45 FF                                      inc     [ebp+buf_index_cp]
    .text:0040A793 88 14 38                                      mov     [eax+edi], dl
    .text:0040A796 0F B6 45 FF                                   movzx   eax, [ebp+buf_index_cp]
    .text:0040A79A 0F BE 16                                      movsx   edx, byte ptr [esi]
    .text:0040A79D 80 C1 02                                      add     cl, 2
    .text:0040A7A0 3B C2                                         cmp     eax, edx
    .text:0040A7A2 7C E6                                         jl      short loc_40A78A
    .text:0040A7A4
    .text:0040A7A4                               loc_40A7A4:
    .text:0040A7A4 0F B6 45 FF                                   movzx   eax, [ebp+buf_index_cp]
    .text:0040A7A8 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:0040A7AC 80 3F 57                                      cmp     byte ptr [edi], 57h
    .text:0040A7AF 75 34                                         jnz     short loc_40A7E5
    .text:0040A7B1 8B 83 20 40 41 00                             mov     eax, g_encrypted_strings[ebx]
    .text:0040A7B7 0F B6 00                                      movzx   eax, byte ptr [eax]
    .text:0040A7BA 8D 4F 01                                      lea     ecx, [edi+1]
    .text:0040A7BD 48                                            dec     eax
    .text:0040A7BE 51                                            push    ecx
    .text:0040A7BF E8 59 48 00 00                                call    Str__xToUnicode
    .text:0040A7C4 8B 0D F4 4A 41 00                             mov     ecx, g_decrypted_strings
    .text:0040A7CA 57                                            push    edi
    .text:0040A7CB 89 04 0B                                      mov     [ebx+ecx], eax
    .text:0040A7CE E8 11 44 00 00                                call    Mem__free
    .text:0040A7D3 A1 F4 4A 41 00                                mov     eax, g_decrypted_strings
    .text:0040A7D8 83 3C 03 00                                   cmp     dword ptr [ebx+eax], 0
    .text:0040A7DC 75 10                                         jnz     short loc_40A7EE
    .text:0040A7DE
    .text:0040A7DE                               loc_40A7DE:
    .text:0040A7DE
    .text:0040A7DE 32 C0                                         xor     al, al
    .text:0040A7E0 E9 CA 00 00 00                                jmp     loc_40A8AF
    .text:0040A7E5
    .text:0040A7E5                               loc_40A7E5:
    .text:0040A7E5 A1 F4 4A 41 00                                mov     eax, g_decrypted_strings
    .text:0040A7EA 47                                            inc     edi
    .text:0040A7EB 89 3C 03                                      mov     [ebx+eax], edi
    .text:0040A7EE
    .text:0040A7EE                               loc_40A7EE:
    .text:0040A7EE FF 45 F8                                      inc     [ebp+index]
    .text:0040A7F1 66 83 7D F8 67                                cmp     word ptr [ebp+index], 67h
    .text:0040A7F6 0F 82 5D FF FF FF                             jb      loc_40A759
    ...

In [4]:
def decrypt_string(key, enc_string_addr):
    enc_string_addr_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    # first byte is a length
    str_len = struct.unpack("B", pe.get_data(enc_string_addr_rva, 1))[0]
    enc_string = pe.get_data(enc_string_addr_rva + 1, str_len)
    
    plain_string = b""
    for eb in enc_string:
        pb = (key + eb) & 0xff
        plain_string += struct.pack("B", pb)
        key += 2
        
    plain_string = plain_string.decode()
    
    # first byte is W or A for wide or ascii
    plain_string = plain_string[1:]
    
    return plain_string


match = re.search(rb'\xc1\xe3\x02\x8b\x83(?P<enc_strings_addr>[\s\S]{4}).*?\x33\xc0\xb1(?P<key>[\s\S]).*?\x66\x83\x7d\xf8(?P<num_entries>[\s\S])', pe.get_memory_mapped_image())
enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
key = struct.unpack("B", match.group("key"))[0]
num_entries = struct.unpack("B", match.group("num_entries"))[0]

plain_strings = {}
for i in range(num_entries):
    offset = enc_strings_addr + 4 * i
    offset_rva = offset - pe.OPTIONAL_HEADER.ImageBase
    enc_string_addr = pe.get_dword_at_rva(offset_rva)
    plain_string = decrypt_string(key, enc_string_addr)
    print(f"{i}: {plain_string}")
    plain_strings[i] = plain_string

0: lowsec
1: user.ds
2: local.ds
3: sdra64.exe
4: winlogon.exe
5: svchost.exe
6: explorer.exe
7: SYSTEM
8: _AVIRA_2110
9: _AVIRA_2101
10: _AVIRA_2108
11: _AVIRA_2109
12: _AVIRA_21099
13: software\microsoft\windows nt\currentversion\network
14: UID
15: software\microsoft\windows nt\currentversion\winlogon
16: software\microsoft\windows\currentversion\run
17: userinit
18: software
19: system
20: csrss.exe
21: %08X%08X%08X%X
22: %s_%08X
23: ntdll.dll
24: outpost.exe
25: zlclient.exe
26: SetErrorMode
27: *%u.%u.%u.%u*
28: image/jpeg
29: screens\%s\%04X_%08X.jpg
30: winsta0
31: default
32: gdiplus.dll
33: ole32.dll
34: gdi32.dll
35: DISPLAY
36: GdiplusStartup
37: GdiplusShutdown
38: GdipCreateBitmapFromHBITMAP
39: GdipDisposeImage
40: GdipGetImageEncodersSize
41: GdipGetImageEncoders
42: GdipSaveImageToStream
43: CreateStreamOnHGlobal
44: CreateDCA
45: CreateCompatibleDC
46: GetDeviceCaps
47: CreateCompatibleBitmap
48: SelectObject
49: BitBlt
50: DeleteObject
51: DeleteDC
52: reboot
53: shu

## Extract Version

Pseudocode:

    ...
    version[0] = 0x1020713;
    r = BinStorage::_addItem(binStorage, 10003, 4, version);
    ...
    
Disassembly:

    ...
    .text:00412819 BB 13 27 00 00                                mov     ebx, 2713h
    .text:0041281E 8B C6                                         mov     eax, esi
    .text:00412820 C7 45 F8 13 07 02 01                          mov     [ebp+version], 1020713h
    ...

In [5]:
match = re.search(rb'\xbb\x13\x27\x00\x00[\s\S]{2}?\xc7\x45[\s\S](?P<version>[\s\S]{4})', 
                  pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

1.2.7.19


## Get BaseConfig

Pseudocode:

    ... 
    pe = (g_self_image + *(g_self_image + 0x3C));
    data1_section_addr = 0;
    if ( pe->FileHeader.NumberOfSections >= 4u )
    {   
        data1_section_header = (&pe->OptionalHeader.DataDirectory[3] + pe->FileHeader.SizeOfOptionalHeader);
        data1_section_addr = (g_self_image + data1_section_header->VirtualAddress);
        if ( IsBadReadPtr(data1_section_addr, data1_section_header->Misc.PhysicalAddress) )
            data1_section_addr = 0;
    }   
    g_base_config = data1_section_addr;
    ... 

In [6]:
data1_section = pe.sections[3]
base_config = pe.get_data(data1_section.VirtualAddress, data1_section.Misc_VirtualSize)
print(base_config)  

b'j\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\xee6\x00`\xea\x00\x00`\xea\x00\x00`\xea\x00\x00\x80O\x12\x00`\xea\x00\x00!\x00\x1d\x00\x00\x04w\xdb\xb5\xbc(\x9f\x10G\xfel\xf8F\xf9\x11\xd1RZ\x15\xc5H\xd3^\xfbT\xacs\xf6xpSU\x9e7\xbb\xef\x166\x90\x93*\x18\xedCP\xf3\x92\x19\x14u\xc0\xabA\x8a\x0ci\x91\x06\x1a2\th\xa9\xae8\xa1O\\\'\x9bJ\xb1\r\xf2&\xf4\xcc\x82}30.\x7f"j\x0b\xb9<\xd9X)q\xa2\x84N|\xaf\xd5\xa3Q\x96\xc1\x88\x9a\xc4\xdc\x98D\xb6;\x80\xcb\x9d5\xc3\xee@\nm\xc2\x02\xcd={\x17\xdd-\x8d\x81y\xb7\xb0\x8c\x8fb\x08\xa7\x1f\xf0\xf7\x94\xd2\xe8\xb2\xfc\x95I\xcfknro\xa8~\xda\xd8\xe9+W\xd6\x1c\x05!>\xb3\xe1\xa6\x13\x12/\x03_,\xc9\xbd`L\x839\xca\xbf\xea?\xe5\xb4B\xf5%\xaa\xc6\xa5\xe2tv\xbeV4\xeb:\xe6\x99\x1d\xd0Y\x04M\xc7\xd7[\xd4\xb8\x8e\x97\xad\xba\x1b\xde\x9c\xa4\xecd\x89\xf1z\x87\xdf\xc8\x01cf\x85$\x8b] \xe0a\xce1#E\xa0K\xe7\xfag\xe3\xe4\xff\x86\x0f\x00\xfd\x0ee\x1e\x07\x00\x00^}f}(@\x19wX\x87U\x8cS\x8f>\x98\x04\x8c5\\3\xa9\xf9\x985\xa7(\xa6%o\x1c\xae$^}f}(@\x19wX\x87U\x8cS\x8f>\x98\x04

## Parse BaseConfig

Pseudocode:

    void *__userpurge decrypt_url@<eax>(int plainbuf@<eax>, unsigned int len@<ebx>, int encbuf)
    {
      int *p_buf; // edi
      void *buf_len; // eax
      unsigned int i; // eax
      int buf; // ecx

      p_buf = plainbuf;
      if ( !plainbuf )
      {
        p_buf = &encbuf;
    LABEL_7:
        for ( i = 0; i < len; ++i )
        {
          buf = *p_buf;
          if ( (i & 1) != 0 )
            *(buf + i) += -7 - 2 * i;
          else
            *(i + buf) += 2 * (i + 5);
        }
        return len;
      }
      buf_len = (len + 16);
      if ( len != -16 )
        buf_len = Mem::alloc(buf_len);
      *p_buf = buf_len;
      if ( buf_len )
      {
        Mem::_copy(buf_len, encbuf, len);
        goto LABEL_7;
      }
      return buf_len;
    }

In [7]:
def decrypt_url(encrypted_url):
    url = b""
    for i, eb in enumerate(encrypted_url):
        if i & 1 == 1:
            pb = (eb - 7 - 2 * i) & 0xff
        else:
            pb = (eb + 2 * (i + 5)) & 0xff
            
        url += struct.pack("B", pb)
        
    return url


defaultBotnet = base_config[0x8:8+4].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delayConfig = struct.unpack("I", base_config[0xc:0xc+4])[0]
print(f"delayConfig: {delayConfig}\n")
delayConfig2 = struct.unpack("I", base_config[0x10:0x10+4])[0]
print(f"delayConfig2: {delayConfig2}\n")

delayReport = struct.unpack("I", base_config[0x14:0x14+4])[0]
print(f"delayReport: {delayReport}\n")
delayReport2 = struct.unpack("I", base_config[0x18:0x18+4])[0]
print(f"delayReport2: {delayReport2}\n")

delayStats = struct.unpack("I", base_config[0x1c:0x1c+4])[0]
print(f"delayStats: {delayStats}\n")
delayStats2 = struct.unpack("I", base_config[0x20:0x20+4])[0]
print(f"delayStats2: {delayStats2}\n")

baseKey = base_config[0x2a:0x2a+258]
print(f"baseKey: {baseKey}\n")

defaultConfig_len = base_config[0x24]
print(f"defaultConfig_len: {defaultConfig_len}\n")

encrypted_defaultConfig = base_config[0x12c:0x12c+defaultConfig_len]
defaultConfig = decrypt_url(encrypted_defaultConfig).decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

testDownloadDelay_url_len = base_config[0x26]
print(f"testDownloadDelay_url_len: {testDownloadDelay_url_len}\n")

offset = 0x12c + defaultConfig_len
encrypted_testDownloadDelay_url = base_config[offset:offset+testDownloadDelay_url_len]
testDownloadDelay_url = decrypt_url(encrypted_testDownloadDelay_url).decode().replace("http", "hxxp")
print(f"testDownloadDelay_url: {testDownloadDelay_url}\n")

testDownloadDelay_buffer_len = struct.unpack("H", base_config[0x28:0x28+2])[0]
print(f"testDownloadDelay_buffer_len: {testDownloadDelay_buffer_len}\n")

num_langids = base_config[0x25]
print(f"num_langids: {num_langids}\n")

offset = 0x12c + defaultConfig_len + testDownloadDelay_url_len
langids = base_config[offset:offset+num_langids*2]
print(f"langids: {langids}\n")

defaultBotnet:     

delayConfig: 3600000

delayConfig2: 60000

delayReport: 60000

delayReport2: 60000

delayStats: 1200000

delayStats2: 60000

baseKey: b'w\xdb\xb5\xbc(\x9f\x10G\xfel\xf8F\xf9\x11\xd1RZ\x15\xc5H\xd3^\xfbT\xacs\xf6xpSU\x9e7\xbb\xef\x166\x90\x93*\x18\xedCP\xf3\x92\x19\x14u\xc0\xabA\x8a\x0ci\x91\x06\x1a2\th\xa9\xae8\xa1O\\\'\x9bJ\xb1\r\xf2&\xf4\xcc\x82}30.\x7f"j\x0b\xb9<\xd9X)q\xa2\x84N|\xaf\xd5\xa3Q\x96\xc1\x88\x9a\xc4\xdc\x98D\xb6;\x80\xcb\x9d5\xc3\xee@\nm\xc2\x02\xcd={\x17\xdd-\x8d\x81y\xb7\xb0\x8c\x8fb\x08\xa7\x1f\xf0\xf7\x94\xd2\xe8\xb2\xfc\x95I\xcfknro\xa8~\xda\xd8\xe9+W\xd6\x1c\x05!>\xb3\xe1\xa6\x13\x12/\x03_,\xc9\xbd`L\x839\xca\xbf\xea?\xe5\xb4B\xf5%\xaa\xc6\xa5\xe2tv\xbeV4\xeb:\xe6\x99\x1d\xd0Y\x04M\xc7\xd7[\xd4\xb8\x8e\x97\xad\xba\x1b\xde\x9c\xa4\xecd\x89\xf1z\x87\xdf\xc8\x01cf\x85$\x8b] \xe0a\xce1#E\xa0K\xe7\xfag\xe3\xe4\xff\x86\x0f\x00\xfd\x0ee\x1e\x07\x00\x00'

defaultConfig_len: 33

defaultConfig: hxxp://brnsounds.cc/ex/config.bin

testDownloadDelay_url_le

## Download and Decrypt DynamicConfig

![](zeus1_1.2.7.19_pcap.png)

An encrypted DynamicConfig is archived on the [Internet Archive](https://web.archive.org/web/20120730050214/https://zeustracker.abuse.ch/monitor.php?show=config&hash=a12d962f53c2f0e80a6d620c3fa9408b&downloadfile=1).

Pseudocode:

    ... 
    if ( rc4Key )
        Mem::_copy(rc4Key_cp, rc4Key, 0x102u);
    if ( a2 == v5 )
    {   
        if ( rc4Key != v5 )
            Crypt::_rc4(rc4Key_cp, binStorage, dataSize);
    ... 

In [8]:
def rc4_keystate(S, in_buf):
    out_buf = b""

    i = S[256]
    j = S[257]
    
    for in_byte in in_buf:
        (i, j, S, K) = rc4_prga(i, j, S)
        out_byte = in_byte ^ K
        out_buf += struct.pack("B", out_byte)

    return out_buf


def rc4_prga(i, j, S):
    i = (i + 1) % 256
    j = (j + S[i]) % 256

    S = bytearray(S)
    S[i], S[j] = S[j], S[i]
    S = bytes(S)

    K = S[(S[i] + S[j]) % 256]

    return (i, j, S, K)


# Load encrypted DynamicConfig
fp = open("ZeuS_config_a12d962f53c2f0e80a6d620c3fa9408b.bin", "rb")
config_bin = fp.read()
fp.close()

binstorage = rc4_keystate(baseKey, config_bin)
print(binstorage)

b'\x10\x89\x00\x00\x00\x00\x00\x00v\x00\x00\x00\x03\xf0\x92\xb4\xce\xfc\xea\xd1|\x00\xabu\x8f\x1d\xa6|!N\x00\x00\x00\x00\x00 \x04\x00\x00\x00\x04\x00\x00\x00\x13\x07\x02\x01"N\x00\x00\x00\x00\x00 \x1c\x00\x00\x00\x1c\x00\x00\x00http://brnsounds.cc/ex/1.exe#N\x00\x00\x00\x00\x00  \x00\x00\x00 \x00\x00\x00http://brnsounds.cc/.zs/gate.php&N\x00\x00\x01\x00\x00 \x85\x00\x00\x00\xa1\x00\x00\x00\xdb\xff\xff\xff!*.microsoft.com/*\x00!http:/\tm\xd6}\xed\xfdyspace\x16\x15\x14s\x15w\x00\xb0\xbf\xbd\xfd.grup1ant\x02der.es5\xc0\xda\xff?odnoklassniki.)\x1b\xfb?\xbb{vko3kte\x16@*/login.K0`Ovmp\x12atl\x10\x00\x00\x00\x00\x00\x00\x00H\x00\xff*N\x00\x00\x01\x00\x00 \xc0\x00\x00\x00\xe8\x00\x00\x00\xff\xbf\xfc\xffO\x00\x01\x00\x03\x01\x06\x00\x10\x00<\x00D\x00https://banki\xcf\xfd\xff\xffng.*.de/cgi/ueberweisu\x15\x10\xd7}{\xff*\x00*&tid=\x07\x1atrag\nIB\xb6%\xf9o\x03\x00N9\x00;\x00=Jt=`\xef\xc3\xdan%VgadX\x0e\xef\xb7\xdf\xbdK\x01KktNrT\x12Enz\x00PH[\x1b\xb0\x9f\x02A\x00N\x97w\x00}i\xff\xdb\xdfkw9A*/j\n

## Parse DynamicConfig

In [9]:
def parse_binstorage_header(binstorage):
    header = {
        "size": struct.unpack("I", binstorage[0:4])[0],
        "flags": struct.unpack("I", binstorage[4:8])[0],
        "count": struct.unpack("I", binstorage[8:12])[0],
        "md5Hash": binstorage[12:28],
    }
        
    return header


def parse_binstorage_sections(section_data, num_sections):
    CFGIDS = {
        20001: "CFGID_LAST_VERSION",
        20002: "CFGID_LAST_VERSION_URL",
        20003: "CFGID_URL_SERVER_0",
        20004: "CFGID_URL_ADV_SERVERS",
        20005: "CFGID_HTTP_FILTER",
        20006: "CFGID_HTTP_BOTLOG_FILTER",
        20007: "CFGID_HTTP_POSTDATA_FILTER",
        20008: "CFGID_HTTP_FAKES_LIST",
        20009: "CFGID_HTTP_INJECTS_LIST",
        20010: "CFGID_HTTP_TANGRABBER_LIST",
        20011: "CFGID_DNS_LIST"
    }
    
    sections = []
    offset = 0
    for i in range(num_sections):
        section = {}
        
        id_ = struct.unpack("I", section_data[offset:offset+4])[0]
        if int(id_) in CFGIDS:
            section["id"] = f'{CFGIDS[int(id_)]} ({id_})'
        else:
            section["id"] = id_
            
        section["flags"] = struct.unpack("I", section_data[offset+4:offset+8])[0]
        section["size"] = struct.unpack("I", section_data[offset+8:offset+12])[0]
        section["realSize"] = struct.unpack("I", section_data[offset+12:offset+16])[0]
        # additional parsing of data is left as an exercise
        section["data"] = section_data[offset+16:offset+16+section["size"]]
        
        offset += 4 + 4 + 4 + 4 + section["size"]
        
        sections.append(section)
        
    return sections


binstorage_header = parse_binstorage_header(binstorage)
print("binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[28:], binstorage_header["count"])
print()
print("binstorage_sections:\n")
pprint.pprint(binstorage_sections)

binstorage_header:

{'count': 118,
 'flags': 0,
 'md5Hash': b'\x03\xf0\x92\xb4\xce\xfc\xea\xd1|\x00\xabu\x8f\x1d\xa6|',
 'size': 35088}

binstorage_sections:

[{'data': b'\x13\x07\x02\x01',
  'flags': 536870912,
  'id': 'CFGID_LAST_VERSION (20001)',
  'realSize': 4,
  'size': 4},
 {'data': b'http://brnsounds.cc/ex/1.exe',
  'flags': 536870912,
  'id': 'CFGID_LAST_VERSION_URL (20002)',
  'realSize': 28,
  'size': 28},
 {'data': b'http://brnsounds.cc/.zs/gate.php',
  'flags': 536870912,
  'id': 'CFGID_URL_SERVER_0 (20003)',
  'realSize': 32,
  'size': 32},
 {'data': b'\xdb\xff\xff\xff!*.microsoft.com/*\x00!http:/\tm\xd6}\xed\xfdyspa'
          b'ce\x16\x15\x14s\x15w\x00\xb0\xbf\xbd\xfd.grup1ant\x02der.e'
          b's5\xc0\xda\xff?odnoklassniki.)\x1b\xfb?\xbb{vko3kte\x16@*/login.K'
          b'0`Ovmp\x12atl\x10\x00\x00\x00\x00\x00\x00\x00H\x00\xff',
  'flags': 536870913,
  'id': 'CFGID_HTTP_BOTLOG_FILTER (20006)',
  'realSize': 161,
  'size': 133},
 {'data': b'\xff\xbf\xfc\xffO\x00\x01\x

          b'\xb1\xcem\x87wviu\x0ee)2\xb7[\xfb`\x0fEntr\xa7Non\x98ic'
          b'\xec\xdb\xd6\xb6k\x10j\xde/c\xb9pt:2(\xbfu\xb7m"\xd9\xad\x02- \x00F'
          b'\x16day, \xff\xdb\x85\xf6Octo\xf3219\x0b2007 3:02\xfb\xd6\x1e\xfb'
          b' AM \x101CheIw();\x0f\x9d\x1d;l6O\xb2\xc5\x14fu\xd6&\xbb\xb0qt'
          b'\x1fn -{xR[lk\xffvv=docu\xb9\x97.\x9bp.\x85\xedcg\xc2.\xb1;if(#'
          b".5&\xbc\xa1\xfdngth<44@rt('\x96\x17\xc2\xad\xdd\xfersR\x12 (\xa6)"
          b"\x1e+\xb4\x0b\x0co 2\x96Qo.'\xd6]\xf6\xdf\x9areturn;}mx"
          b'\x1bn\x15\xb6\xb5\xd6\xdaP\x85\x97\x13?!sz\xd0\xb5o\x13=w;Va'
          b'c\xb3\xd8}\x00\x00\x00\x00\x00\x00H\x00\xff',
  'flags': 2147483649,
  'id': 24,
  'realSize': 622,
  'size': 441},
 {'data': b'\xfd\xff\xff\xcb \x00\x06\x00name="oppasswd"*</tr>\xb1c\xb1'
          b'\xae\x05\x04\x0b\x1f\x0f\x19\x01\n\xdb\xf6\xb7\xfftd height024" a'
          b'l\x0bn\nro7w{\x11\rclA\rfi\x071">\xd6\xbe\x9d\xfd&nbsp;\x05C\x11ve'
          b' d\x02\x1b+\xec\xbd\xfbr

 {'data': b'\x7f\xfb\xff\xcb%\x00\x06\x00/Jaen/images\x06nterro\xff'
          b'\xfd\xfe\xddga\x07.jpg*>\x06\x00\x00\x04\x01*</TR>c\xff\x1f\xf6 '
          b'\r\n<\x06D height=29\x10P\xff\xb6\xfd\xff><FONT face="VJdHa, Tah'
          b'om\xff\xffo_" color=#007042 siz$2\xb2\xcd\xb9\xb71/2\x06<_DX\xb6'
          b'}\x15\xb6RFir\x1b:X\xd6\xbe%KT\x0bINPUK\x1c\xbd\x9b\xfd\xedxL'
          b'\xf3gth=103\x07valu\xd8\xbfm\xcdaR^m\x07ESpass>C\xd9\x0b#\x1c\x19.'
          b'<"put$\x8f\x9d0\xc2"5""r\x1e On\xff\xffv\xfbClick\x1bjaQscript: '
          b'if (doc\xd9\xb6-\xb4u3W.\xb0gC.`\xf6\x8f\xb5\xd8.u\x12\x8d < 5'
          b") {\xaav\x8b\xbd\xb5\x13\xe6t('\xc6io \x1d\xfe\x7f\xbb\xb5\xe68raB'"
          b');return;};"\x00\x00\x00\x00\x00\x00\t\x00\xff',
  'flags': 2147483649,
  'id': 55,
  'realSize': 448,
  'size': 325},
 {'data': b'\xe7\xff\xff\xcb\x16\x00\x06\x00name="PAN"*</td>\x06\x00\xdf\xdc'
          b'w\xb1\xe2\x1b\x10r>\r\n<\x05\x03d cl\x8bm\xff\xedass+sp5">&nb'
          b'\x07;1%\xdb

 {'data': b'\x1b\x00\x00\x00\x06\x00<span id="money_sum">\r\x00\x00\x00\x06'
          b'\x00</span>\x06\x00\x00\x00\x00\x00',
  'flags': 2147483648,
  'id': 96,
  'realSize': 46,
  'size': 46},
 {'data': b'&\x00\x00\x00\x06\x00<h3 id="LI6BTHC" class="frmHdr">\x15\x00'
          b'\x00\x00\x06\x00name="txnID" />\x06\x00\x00\x00\x00\x00',
  'flags': 2147483648,
  'id': 97,
  'realSize': 65,
  'size': 65},
 {'data': b'C\x00\x00\x00\x06\x00<td class="actionHeaderTopPadding" >Balances a'
          b'nd Transactions*\x00\x00\x00\x06\x00<!--This is required for bway:b'
          b'utton\x06\x00\x00\x00\x00\x00',
  'flags': 2147483648,
  'id': 98,
  'realSize': 115,
  'size': 115},
 {'data': b'\x18\x00\x00\x00\x06\x00Passnumber:</SPAN>0\x00\x00\x00\x06\x00<T'
          b'D COLSPAN="4" CLASS="consolebackground">\x06\x00\x00\x00\x00\x00',
  'flags': 2147483648,
  'id': 99,
  'realSize': 78,
  'size': 78},
 {'data': b'@\x00\x00\x00\x06\x00<span OnClick="window.open(\'https://www.nw'
          b'

## Commands

Pseudocode:
    
    ... 
    ci = 0;
    while ( Str::_CompareA(-1, v8, *(&g_s416->lowsec + g_command_offset[4 * ci]), v23) )
    {   
        if ( ++ci >= 25u )
            goto LABEL_30;
    }   
    ... 
    
Disassembly:

    .text:00407E52 C6 45 FD 00                                   mov     [ebp+ci], 0
    .text:00407E56
    .text:00407E56                               loc_407E56:
    .text:00407E56 0F B6 45 FD                                   movzx   eax, [ebp+ci]
    .text:00407E5A 0F B7 0C C5 F8 41 41 00                       movzx   ecx, g_command_offset[eax*8]
    .text:00407E62 8B 3D F4 4A 41 00                             mov     edi, g_s416
    .text:00407E68 FF 75 E4                                      push    [ebp+var_1C]
    .text:00407E6B 8B 0C 8F                                      mov     ecx, [edi+ecx*4]
    .text:00407E6E 83 C8 FF                                      or      eax, 0FFFFFFFFh
    .text:00407E71 8B D6                                         mov     edx, esi
    .text:00407E73 E8 18 72 00 00                                call    Str___CompareA
    .text:00407E78 85 C0                                         test    eax, eax
    .text:00407E7A 74 0E                                         jz      short loc_407E8A
    .text:00407E7C FE 45 FD                                      inc     [ebp+ci]
    .text:00407E7F 80 7D FD 19                                   cmp     [ebp+ci], 19h
    .text:00407E83 72 D1                                         jb      short loc_407E56
    .text:00407E85 E9 AD 00 00 00                                jmp     loc_407F37

In [10]:
match = re.search(rb'\x0f\xb7\x0c\xc5(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(25):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    string_index = pe.get_word_at_rva(index_rva)
    
    print(f"{i}: {plain_strings[string_index]}")

0: block_fake
1: unblock_fake
2: bc_add
3: bc_del
4: block_url
5: unblock_url
6: addsf
7: delsf
8: getfile
9: getcerts
10: resetgrab
11: upcfg
12: kbot
13: rename_bot
14: getmff
15: delmff
16: sethomepage
17: kos
18: reboot
19: shutdown
20: rexeci
21: rexec
22: lexeci
23: lexec
24: resetkwm
